In [1]:
import os

import pandas as pd
import polars as pl

import kaggle_evaluation.cmi_inference_server

import numpy as np
import sklearn as sk

The evaluation API requires that you set up a server which will respond to inference requests. We have already defined the server; you just need write the predict function. When we evaluate your submission on the hidden test set the client defined in the gateway will run in a different container with direct access to the hidden test set and hand off the one sequence at a time.

Your code will always have access to the published copies of the files.

In [2]:
'''
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    # Replace this function with your inference code.
    # You can return either a Pandas or Polars dataframe, though Polars is recommended.
    # Each prediction (except the very first) must be returned within 30 minutes of the batch features being provided.
    return 'Text on phone'
'''

"\ndef predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:\n    # Replace this function with your inference code.\n    # You can return either a Pandas or Polars dataframe, though Polars is recommended.\n    # Each prediction (except the very first) must be returned within 30 minutes of the batch features being provided.\n    return 'Text on phone'\n"

In [3]:
'''
#Checking how test data looks
file_path = '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv'

# Reading the CSV file
df_test_data = pd.read_csv(file_path)

# Get number of rows in test data
num_rows = df_test_data.shape[0]
print('Number of row in test data = ', num_rows)

# Display the first few rows of the dataframe, we will be doing this as checking step every step along the way
df_test_data.head(5)

# Investigating further I found thet data that goes into the predict function is different
'''

"\n#Checking how test data looks\nfile_path = '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv'\n\n# Reading the CSV file\ndf_test_data = pd.read_csv(file_path)\n\n# Get number of rows in test data\nnum_rows = df_test_data.shape[0]\nprint('Number of row in test data = ', num_rows)\n\n# Display the first few rows of the dataframe, we will be doing this as checking step every step along the way\ndf_test_data.head(5)\n\n# Investigating further I found thet data that goes into the predict function is different\n"

In [4]:
# Making the Model
    
# Loading the CSV file
file_path = '/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv'
   
# Reading the CSV file
df_original_data = pd.read_csv(file_path)

# Display the first few rows of the dataframe, we will be doing this as checking step every step along the way
#df_original_data.head()

# Checking number of null values in each column
#df_original_data.isnull().sum()

# Defining feature and target
A = df_original_data.iloc[:, 9:].values
b = df_original_data.iloc[:, 8].values

#print(A)
#print(b)

# Now the feature column has some null values - imputing them with a constant value Zero    
# Replace NaNs with 0
A = df_original_data.iloc[:, 9:].values
A = np.nan_to_num(A, nan=0)
#print(A[:5])  # Display first 5 rows

# Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
A_train_s = sc.fit_transform(A)

# Instead of test train split, using whole of the data to train our model.

# Fitting KNN to the training set

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(A_train_s, b)

KNeighborsClassifier()

In [5]:
# Here first row from test data is taken and prediction returned, 
# the second time the funtion is called the second row of test data is taken and prediction returned and so on.

# Global counter to keep track of which test row to use
current_test_index = 0
program_terminate = 0

def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    global current_test_index
    global program_terminate

    # Code block for when the funtion reaches the end of test data
    #print('Number of row in test data =', len(sequence)) 
    # Sneaky little program is using two sequences one of 51 lenght and the other 56 length
    if current_test_index >= len(sequence):
        print("Reached end of test data.")
        program_terminate = 1
        return 'Cheek - pinch skin' # Dummy return - will reduce accuracy / score a bit, oh well.


    # Importing the testing data 
    # Convert Polars DataFrame to Pandas 
    df_original_test_data = sequence.to_pandas()

    # Display the first few rows of the dataframe, we will be doing this as checking step every step along the way
    #df_original_test_data.head()

    #print(current_test_index)
    
    # Only taking needed columns from test data, and the needed row
    original_test_data = df_original_test_data.iloc[current_test_index, 4:].values

    # To show current row_id, sequence_id, sequence_counter data being tested out.
    row_id = df_original_test_data['row_id'].iloc[current_test_index]
    sequence_id = df_original_test_data['sequence_id'].iloc[current_test_index]
    sequence_counter = df_original_test_data['sequence_counter'].iloc[current_test_index]
    #print(row_id, ",", sequence_id, ",", sequence_counter) 
    
    current_test_index += 1 # To update the row value, when the funtion is called next time
    
    # Reshape is done coz above code gives a 1D shape while the input for next is a 2D shape, just a small tweak.
    original_test_data = original_test_data.reshape(1, -1)
    #print(original_test_data)

    # Predicting the test set - results
    y_pred = classifier.predict(original_test_data)
    #print(y_pred)

    y_pred_string = str(y_pred[0]) # Converting the return value to a string.
    #print(y_pred_string)
    #print(sequence_id, y_pred_string)
    print(row_id, ",", sequence_id, ",", sequence_counter, ",", y_pred_string)
    
    #print('Checkpoint 1 ~ Akash')
    return y_pred_string

    # ~ End of code.

In [6]:
'''
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )
'''

"\ninference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)\n\nif os.getenv('KAGGLE_IS_COMPETITION_RERUN'):\n    inference_server.serve()\nelse:\n    inference_server.run_local_gateway(\n        data_paths=(\n            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',\n            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',\n        )\n    )\n"

In [7]:
print('row_id, sequence_id, sequence_counter, gesture prediction')

# Call predict() for each row
for i in range(200): 
    #print('Loop number = ', i)

    if program_terminate == 1:
        break
        
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)
    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        print('Checkpoint 2 ~ Akash')
        inference_server.serve()
    else:
        #print('Checkpoint 2.5 ~ Akash')
        inference_server.run_local_gateway(
            data_paths=(
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
            )
        )
    #print('Checkpoint 3 ~ Akash')

row_id, sequence_id, sequence_counter, gesture prediction
SEQ_000001_000000 , SEQ_000001 , 0 , Wave hello
SEQ_000011_000001 , SEQ_000011 , 1 , Pull air toward your face
SEQ_000001_000002 , SEQ_000001 , 2 , Wave hello
SEQ_000011_000003 , SEQ_000011 , 3 , Text on phone
SEQ_000011_000004 , SEQ_000011 , 4 , Text on phone
SEQ_000001_000005 , SEQ_000001 , 5 , Wave hello
SEQ_000011_000006 , SEQ_000011 , 6 , Text on phone
SEQ_000001_000007 , SEQ_000001 , 7 , Wave hello
SEQ_000011_000008 , SEQ_000011 , 8 , Text on phone
SEQ_000001_000009 , SEQ_000001 , 9 , Cheek - pinch skin
SEQ_000011_000010 , SEQ_000011 , 10 , Pull air toward your face
SEQ_000001_000011 , SEQ_000001 , 11 , Wave hello
SEQ_000001_000012 , SEQ_000001 , 12 , Wave hello
SEQ_000011_000013 , SEQ_000011 , 13 , Cheek - pinch skin
SEQ_000011_000014 , SEQ_000011 , 14 , Wave hello
SEQ_000001_000015 , SEQ_000001 , 15 , Cheek - pinch skin
SEQ_000001_000016 , SEQ_000001 , 16 , Cheek - pinch skin
SEQ_000011_000017 , SEQ_000011 , 17 , Cheek -